In [99]:
import pandas as pd 
import numpy as np
df = pd.read_csv("Housing.csv")
df.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [100]:
norm = df.copy()
columns = ['price', 'area', 'bedrooms', 'bathrooms']
for column in columns:
  norm[column] = (df[column] - df[column].mean()) / df[column].std()

In [101]:
X = norm[['area', 'bedrooms', 'bathrooms']].values
y = norm['price'].values

In [102]:
def hypothesis_func(X, w):
    return np.dot(X, w)

In [103]:
def loss_func(X, y, w):
    m = len(y)
    y_pred = hypothesis_func(X, w)
    loss = (1 / (2*m)) * np.sum((y_pred - y)**2)
    return loss

In [104]:
def gradient_step(X, y, w, learning_rate):
    m = len(y)
    grad = (X.T @ (hypothesis_func(X, w) - y)) / m
    w -= learning_rate * grad
    return w

In [105]:
def gradient(X, y, learning_rate, num_iter, eps):
    ones = np.ones((X.shape[0], 1))
    X = np.hstack((ones, X))
    w = np.zeros((X.shape[1]))
    loss = loss_func(X, y, w)
    loss_history = [loss]

    for _ in range(num_iter):
        w = gradient_step(X, y, w, learning_rate)

        loss = loss_func(X, y, w)
        if abs(loss - loss_history[-1]) < eps:
            loss_history.append(loss)
            break
        
        loss_history.append(loss)
    return w, loss_history

In [106]:
learning_rate = 0.001
iterations = 100000
eps = 1e-12

w_gradient, losses = gradient(X, y, learning_rate, iterations, eps)

In [107]:
ones = np.ones((X.shape[0], 1))
X = np.hstack((ones, X))
w_analytical = np.linalg.inv(X.T @ X) @ X.T @ y

In [108]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = LinearRegression()
model.fit(X_train, y_train)

intercept = np.array([[model.intercept_]]) 
coef = model.coef_.reshape(-1, 1)

print("Model intercept: ", intercept)
print("Model coefficients: ", coef)

w_sklearn = np.vstack([intercept, coef]) 

Model intercept:  [[-0.01626681]]
Model coefficients:  [[0.        ]
 [0.40082084]
 [0.14213176]
 [0.38208794]]


In [109]:
print("Weights results of gradient descent: ", w_gradient.round(4))
print(f'Analytical Solution: {w_analytical.round(4)}')
print("Weights results of LinearRegression: ", coef.round(4))

Weights results of gradient descent:  [0.     0.4395 0.1606 0.3723]
Analytical Solution: [-0.      0.4395  0.1605  0.3723]
Weights results of LinearRegression:  [[0.    ]
 [0.4008]
 [0.1421]
 [0.3821]]


In [110]:
from sklearn.metrics import mean_squared_error

y_pred_gradient = hypothesis_func(X, w_gradient)
mse_gradient = mean_squared_error(y, y_pred_gradient)
print("MSE of gradient descent method: ", mse_gradient)

y_pred_analytical = hypothesis_func(X, w_analytical)
mse_analytical = mean_squared_error(y, y_pred_analytical)
print("MSE of analytical method: ", mse_analytical)

y_pred_sklearn = model.predict(X_train)
mse_sklearn = mean_squared_error(y_train, y_pred_sklearn)
print("MSE of sklearn LinearRegression method: ", mse_sklearn)

MSE of gradient descent method:  0.51197580290635
MSE of analytical method:  0.5119758013064282
MSE of sklearn LinearRegression method:  0.44611010370502946
